# Hyperparameter tuning Total Moodel

# Tuning a Random Forest Regression Model

Custom built expanding window validation functions will be used 

In [49]:
import pandas as pd
import matplotlib.pyplot as plt

# Import your helper functions
from timeseries_model_utils import (
    expanding_window_splits,
    run_expanding_cv,
    expanding_window_grid_search
)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error


In [53]:
df = pd.read_csv("../data/New_csv.csv")   # replace with your file
df = df.sort_values("Year").reset_index(drop=True)
df.head()


,Unnamed: 0,Year,Homicide,Assault,Sexual offences,Harm or endanger persons,"Robbery, blackmail, and extortion",Burglary,Theft,Fraud and related offences,...,Traffic and vehicle offences,Offences against justice procedures and orders,Offences against government,Environmental offences,Miscellaneous offences,Total,Population,Crime_Index,Total_Lagged,Population_lagged
0,0,1980,353,7208,1401,2176,426,11893,29103,19794,...,173948,27664,1376,3521,9651,331751,3112900,10.657297,331751.0,3112900.0
1,1,1981,366,7933,1092,2299,494,12974,29654,25007,...,142990,17981,2027,3360,9836,294409,3124900,9.421389,331751.0,3112900.0
2,2,1982,329,7760,1298,2231,504,14387,32682,26131,...,104739,13957,1203,4025,7878,252119,3156100,7.988308,294409.0,3124900.0
3,3,1983,407,8741,1449,2693,657,13851,32304,28481,...,105649,15332,736,2252,7774,254658,3199300,7.959804,252119.0,3156100.0
4,4,1984,406,9349,1682,2651,729,13905,33492,28069,...,116414,16389,1300,2235,8552,274022,3227100,8.491277,254658.0,3199300.0


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 24 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Unnamed: 0                                      45 non-null     int64  
 1   Year                                            45 non-null     int64  
 2   Homicide                                        45 non-null     int64  
 3   Assault                                         45 non-null     int64  
 4   Sexual offences                                 45 non-null     int64  
 5   Harm or endanger persons                        45 non-null     int64  
 6   Robbery, blackmail, and extortion               45 non-null     int64  
 7   Burglary                                        45 non-null     int64  
 8   Theft                                           45 non-null     int64  
 9   Fraud and related offences                   

In [57]:
target_col = "Total"        # or any column you want to predict
feature_cols = ["Total_Lagged","Year","Population_lagged"]

X = df[feature_cols]
y = df[target_col]


In [59]:
# param_grid = {
#     "n_estimators": [100, 200, 400],
#     "max_depth": [3, 5, None]
# }

# best_params, best_score, results_df = expanding_window_grid_search(
#     RandomForestRegressor,
#     param_grid,
#     X, y,
#     initial_train_size=10,
#     horizon=1,
#     step=1
# )

# print("\nBest Parameters:", best_params)
# print("Best Score:", best_score)

# results_df



In [60]:

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
    "max_iter": [1000, 5000, 10000]
}


In [61]:

X = X.ffill().bfill()


In [62]:
# from sklearn.linear_model import Lasso

# param_grid = {
#     "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
#     "max_iter": [1000, 5000, 10000],
# }

# best_params, best_score, results_df = expanding_window_grid_search(
#     Lasso,
#     param_grid,
#     X,
#     y,
#     initial_train_size=10,
#     horizon=1,
#     step=1
# )

# print("\nBest Parameters (Lasso):", best_params)
# print("Best Score:", best_score)
# results_df


# train a lasso with alpha 0.001 and max iter 1000 and save it

In [64]:
from sklearn.linear_model import Lasso

# Initialize model
lasso = Lasso(alpha=100, max_iter=1000)

# Train model
lasso.fit(X, y)


Lasso(alpha=100)

In [66]:
lasso.predict(X[-1:])

array([215634.32445086])

In [71]:
X[-1:]

,Total_Lagged,Year,Population_lagged
44,216533.0,2024,5223100.0


# Train Random forest & Save it

In [73]:
rd = RandomForestRegressor(n_estimators=400, max_depth=None)

rd.fit(X, y)

RandomForestRegressor(n_estimators=400)

In [76]:
X[-1:]

,Total_Lagged,Year,Population_lagged
44,216533.0,2024,5223100.0


In [78]:
rd.predict(X[-1:])

array([218474.59])

# Tuning a Gradient Boosting Algorithm

In [82]:
# from sklearn.ensemble import GradientBoostingRegressor

# param_grid = {
#     "n_estimators": [100, 300, 600],          # number of boosting stages
#     "learning_rate": [0.01, 0.05, 0.1],       # shrinkage
#     "max_depth": [1,2, 3, 4],                   # depth of each tree (via base estimator)
#                     # only used when loss="huber" or "quantile"
# }

# best_params, best_score, results_df = expanding_window_grid_search(
#     GradientBoostingRegressor,
#     param_grid,
#     X,
#     y,
#     initial_train_size=10,
#     horizon=1,
#     step=1
# )

# print("\nBest Parameters (GBR):", best_params)
# print("Best Score:", best_score)
# results_df

In [83]:

import joblib


gbr = GradientBoostingRegressor(
    n_estimators=600,
    learning_rate=0.1,
    max_depth=1,
    random_state=42
)

# Train the model
gbr.fit(X, y)


GradientBoostingRegressor(max_depth=1, n_estimators=600, random_state=42)

# Hyperparameter Tuning the Population Model

In [86]:
target_col = "Population"        # or any column you want to predict
feature_cols = ["Total_Lagged","Year","Population_lagged"]

X = df[feature_cols]
y = df[target_col]


In [87]:
# from sklearn.linear_model import Lasso

# param_grid = {
#     "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
#     "max_iter": [1000, 5000, 10000],
# }

# best_params, best_score, results_df = expanding_window_grid_search(
#     Lasso,
#     param_grid,
#     X,
#     y,
#     initial_train_size=10,
#     horizon=1,
#     step=1
# )

# print("\nBest Parameters (Lasso):", best_params)
# print("Best Score:", best_score)
# results_df


In [88]:
gbr = GradientBoostingRegressor(
    n_estimators=600,
    learning_rate=0.1,
    max_depth=1,
    random_state=42
)

In [89]:
lasso_model = Lasso(alpha = 0.001,max_iter = 1000)

In [90]:
lasso_model.fit(X,y)

C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.573e+10, tolerance: 1.896e+09
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.001)

# Recursive Model Function

In [93]:
def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    scaler_X_pop,
    scaler_X_total,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.

    Parameters
    ----------
    X_last : pd.DataFrame
        Last known row of features (t)
    population_model : trained model
        Model that predicts Population_lagged
    total_model : trained model
        Model that predicts total
    scaler_X_pop : fitted scaler
        Scaler used for population model
    scaler_X_total : fitted scaler
        Scaler used for total model
    population_feature_name : str
        Column name of Population_lagged
    horizon : int
        Number of recursive steps

    Returns
    -------
    pd.DataFrame
        Forecasted Population and Total values
    """

    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):

        # ---- Predict Population_lagged ----
        X_pop_scaled = scaler_X_pop.transform(X_current)
        pop_pred = population_model.predict(X_pop_scaled)[0]

        # Update population feature
        X_current[population_feature_name] = pop_pred

        # ---- Predict total ----
        X_total_scaled = scaler_X_total.transform(X_current)
        total_pred = total_model.predict(X_total_scaled)[0]

        # Store predictions
        forecasts.append({
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # ---- Shift lags (if needed) ----
        # Example: Population_lagged_1 → Population_lagged_2
        for col in X_current.columns:
            if "_lag" in col:
                lag_num = int(col.split("_lag")[-1])
                if lag_num > 1:
                    X_current[col.replace(f"_lag{lag_num}", f"_lag{lag_num-1}")] = X_current[col]

        # Set newest lag
        X_current[population_feature_name] = pop_pred

    return pd.DataFrame(forecasts)


In [95]:
gbr.fit(X,y)

GradientBoostingRegressor(max_depth=1, n_estimators=600, random_state=42)

In [110]:
import pandas as pd

def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.

    Parameters
    ----------
    X_last : pd.DataFrame
        Last known row of features (t)
    population_model : trained model
        Model that predicts Population_lagged
    total_model : trained model
        Model that predicts total
    population_feature_name : str
        Column name of Population_lagged
    horizon : int
        Number of recursive steps

    Returns
    -------
    pd.DataFrame
        Forecasted Population and Total values
    """

    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):

        print(X_current)
        # ---- Predict Population_lagged ----
        pop_pred = population_model.predict(X_current)[0]
        print(pop_pred)
        # Update population feature
        X_current[population_feature_name] = pop_pred

        # ---- Predict total ----
        total_pred = total_model.predict(X_current)[0]

        

        # Store predictions
        forecasts.append({
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # # ---- Shift lagged features ----
        # for col in X_current.columns:
        #     if "_lag" in col:
        #         lag_num = int(col.split("_lag")[-1])
        #         if lag_num > 1:
        #             X_current[col.replace(
        #                 f"_lag{lag_num}", f"_lag{lag_num-1}"
        #             )] = X_current[col]

        # # Set newest lag explicitly
        # X_current[population_feature_name] = pop_pred
        X_current[population_feature_name] = pop_pred
        X_current["Total_Lagged"] = total_pred
        X_current["Year"] += 1 

    return pd.DataFrame(forecasts)

In [112]:
predictions = recursive_forecast_with_exogenous(X[-1:], lasso_model, gbr, "Population_lagged")

    Total_Lagged  Year  Population_lagged
44      216533.0  2024          5223100.0
5277296.62856452
    Total_Lagged  Year  Population_lagged
44  5.266385e+06  2025       5.277297e+06
3362451.7015081774
    Total_Lagged  Year  Population_lagged
44  4.318596e+06  2026       3.362452e+06
2083267.5879166853


In [114]:
predictions

,step,Population_lagged_pred,total_pred
0,1,5.277297e+06,5.266385e+06
1,2,3.362452e+06,4.318596e+06
2,3,2.083268e+06,4.136292e+06


In [116]:
X

,Total_Lagged,Year,Population_lagged
0,331751.0,1980,3112900.0
1,331751.0,1981,3112900.0
2,294409.0,1982,3124900.0
3,252119.0,1983,3156100.0
4,254658.0,1984,3199300.0
5,274022.0,1985,3227100.0
6,272657.0,1986,3247100.0
7,298086.0,1987,3246300.0
8,292500.0,1988,3274400.0
9,308817.0,1989,3283400.0


In [124]:
def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.
    """
    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):
        # Predict Population_lagged
        print(population_model.predict(X_current))
        pop_pred = population_model.predict(X_current)
        
        
        # Update population feature
        X_current[population_feature_name] = pop_pred

        # Predict total
        total_pred = total_model.predict(X_current)

        # Store predictions
        current_year = int(X_current["Year"].iloc[0]) 
        forecasted_year = current_year+1
        forecasts.append({
            "Year": forecasted_year,
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # Update features for next iteration
        X_current[population_feature_name] = pop_pred
        X_current["Total_Lagged"] = total_pred
        X_current["Year"] = current_year + 1

    return pd.DataFrame(forecasts)


In [130]:
def predict_total_for_year_recursive(target_year: int, data_file="../data/New_csv.csv"):
    """
    Predict total crime value for a specific year using recursive forecasting.
    
    Args:
        target_year (int): The year to predict total values for
        data_file (str): Path to the dataset file containing lagged features
        
    Returns:
        float: Predicted total value for the target year
    """
    # if not recursive_models_loaded:
    #     if not load_recursive_models():
    #         raise Exception("Failed to load recursive models. Ensure population.joblib and total.joblib exist.")
    
    # Get last known data
    X_last, last_year = get_last_known_data(data_file)
    
    # Validate target year
    if target_year <= last_year:
        raise Exception(f"Target year ({target_year}) must be greater than last known year ({last_year})")
    
    # Calculate forecast horizon
    horizon = target_year - last_year
    
    # Perform recursive forecast
    forecasts = recursive_forecast_with_exogenous(
        X_last=X_last,
        population_model=lasso_model,
        total_model=gbr,
        population_feature_name="Population_lagged",
        horizon=horizon
    )
    print(forecasts)
    
    # Get the prediction for the target year
    #print(forecasts[forecasts["Year"] == target_year]["total_pred"])
    target_prediction = forecasts[forecasts["Year"] == target_year]["total_pred"].iloc[0]
    
    
    return float(target_prediction)

In [132]:
result = predict_total_for_year_recursive(2027)

[5277296.62856452]
[3362451.70150818]
[2083267.58791669]
   Year  step Population_lagged_pred            total_pred
0  2025     1     [5277296.62856452]   [5266385.188041851]
1  2026     2   [3362451.7015081774]   [4318596.170160055]
2  2027     3   [2083267.5879166853]  [4136291.8238027957]


C:\Users\hp\AppData\Local\Temp\ipykernel_14836\2609170765.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(target_prediction)


In [142]:
joblib.dump(lasso_model, "../Models/population.joblib")

['../Models/population.joblib']

In [144]:
joblib.dump(gbr, "../Models/total.joblib")

['../Models/total.joblib']

In [146]:
X

,Total_Lagged,Year,Population_lagged
0,331751.0,1980,3112900.0
1,331751.0,1981,3112900.0
2,294409.0,1982,3124900.0
3,252119.0,1983,3156100.0
4,254658.0,1984,3199300.0
5,274022.0,1985,3227100.0
6,272657.0,1986,3247100.0
7,298086.0,1987,3246300.0
8,292500.0,1988,3274400.0
9,308817.0,1989,3283400.0
